In [1]:
import pathlib
import sys
PATH = pathlib.Path().absolute().resolve()

# %matplotlib inline
import math
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.4)
COLORS = sns.color_palette()

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
def ax_style(ax):
    ax.grid(which='minor', axis='x', dashes=(5, 5), linewidth=.5)
    ax.grid(which='minor', axis='y', dashes=(5, 5), linewidth=.5)
    ax.legend()
    ax.minorticks_on()
    return ax


In [4]:
@interact(
    a=widgets.FloatText(value=8, description="Maximum y"),
    b=widgets.FloatText(value=2, description="Stop x"),
    c=widgets.FloatText(value=2, description="Distance penalty"),
    g=widgets.FloatText(value=1, description="Flat"),
)
def _(a, b, c, g):
    functions = [
        (lambda d: (d + 1) ** 1.3, 'x'),
        (lambda d: (a - 1) * (1 - math.e ** -((d / b) ** 2)) + g + d / c, 'custom'),
    ]
    fig = plt.figure(figsize=(20, 20))

    ax1 = fig.add_subplot(321)
    ax2 = fig.add_subplot(322)
    ax3 = fig.add_subplot(323)
    ax4 = fig.add_subplot(324)
    ax5 = fig.add_subplot(325)
    ax6 = fig.add_subplot(326)
    X = np.linspace(0, 64, 100)
    X_10 = np.linspace(0, 10, 100)
    for i, (fn, name) in enumerate(functions):
        v_fn = np.vectorize(fn)
        Y = v_fn(X)
        Y_10 = v_fn(X_10)
        dY = np.zeros(Y.shape, np.float)
        dY[0:-1] = np.diff(Y) / np.diff(X)
        dY[-1] = (Y[-1] - Y[-2]) / (X[-1] - X[-2])
        ax1.plot(X, Y, label=name, color=COLORS[i % len(COLORS)])
        ax2.plot(X, dY, label=f"({name})`", color=COLORS[i % len(COLORS)])
        ax3.plot(X, 1 / Y, label=name, color=COLORS[i % len(COLORS)])
        ax4.plot(X, 1 / Y, label=name, color=COLORS[i % len(COLORS)])
        ax5.plot(X_10, 1 / Y_10, label=name, color=COLORS[i % len(COLORS)])
        ax6.plot(X_10, 1 / Y_10, label=name, color=COLORS[i % len(COLORS)])

    for ax in (ax1, ax2, ax3, ax4, ax5, ax6):
        ax_style(ax)
        if ax in (ax5, ax6):
            ax.xaxis.set_ticks(np.arange(0, 10 + 1, 1.0))
        else:
            ax.xaxis.set_ticks(np.arange(0, 64 + 1, 8.0))
    ax4.set_yscale("log")
    ax6.set_yscale("log")
    fig.tight_layout()
    plt.show()
    
    print(f"lambda d: {a - 1} * (1 - math.e ** -((d / {b}) ** 2)) + {g} + d / {c}")

interactive(children=(FloatText(value=8.0, description='Maximum y'), FloatText(value=2.0, description='Stop x'…

-----------------------------------------------------------------------------------------------

In [11]:
import requests
import zstandard

from tqdm import tqdm
from requests_futures.sessions import FuturesSession

session = FuturesSession(max_workers=50)
dctx = zstandard.ZstdDecompressor()

matches_url = lambda offset, limit: f"https://api.2018.halite.io/v1/api/match?order_by=desc,game_id&limit={limit}&offset={offset}"
replay_url = lambda game_id: f"https://api.2018.halite.io/v1/api/user/0/match/{game_id}/replay"
limit = int(input("How many replays do you want? "))

data = []
chunk = True
step = 100
i = 0
with tqdm() as progress:
    while chunk and step * i < limit:
        url = matches_url(step * i, step)
        resp = session.get(url)
        try:
            chunk = resp.result().json()
        except json.JSONDecodeError:
            print(resp.text)
            sleep(1)
        else:            
            i += 1
            data.extend(chunk)
            progress.update(step)

replays = []
futures = [session.get(replay_url(item["game_id"])) for item in data]
for item in tqdm(futures):
    raw = item.result().content
    grid = json.loads(dctx.decompress(raw))['production_map']['grid']
    grid = [[x['energy'] for x in row] for row in grid]
    replays.append(grid)
   

How many replays do you want? 500


500it [00:04, 114.12it/s]
100%|████████████████████████████████████████| 500/500 [00:54<00:00, 10.17it/s]


In [12]:
len(replays)

500